In [2]:
# pip install openai

In [36]:
!pip3 show openai | grep Version

Version: 1.63.2


- JSON예쁘게 출력ㅋ

In [37]:
import json
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [38]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import time

- API키 등록

In [51]:
api_key = os.getenv("OPENAI_API_KEY")
assistent_id = os.getenv("ASSISTANT_ID")
client = OpenAI(api_key=api_key)

# 만약 assistant 내용을 수정하고 싶다면?
https://platform.openai.com/playground/assistants
로 들어가 내용을 편집하면 된다.

- 새로운 Thread를 생성하는 함수

In [52]:
def create_new_thread():
    thread = client.beta.threads.create()
    return thread

- thread를 생성하고 메세지를 보내고 Assistant가 User가 질문하였다고 알림을 주는 함수

In [53]:
def summit_message(assistent_id, thread_id, user_message):
    #메세지 전송
    client.beta.threads.messages.create( 
        thread_id=thread_id,
        role="user",
        content=user_message
    )

    #실행시켜 assistant와 연결
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistent_id
    )

    return run

- return 받은 run status는 대기상태. 답변의 상태를 지속적 확인하는 코드
    - run의 상태값
        - queued : 아직 대기열
        - in_progress : 처리중
        - requires_action : 사용자 입력 대기
        - cancelling : 작업 취소중
        - cancelled : 작업 취소 완료
        - failed : 실패(오류)
        - completed : 작업 완료
        - expired : 작업 만료


In [54]:
# run status가 queued or in_progress 상태가 아닐때 까지 지정된 시간간격으로 확인하고 완료되면 값을 반환해주는 함수
def wait_on_run(run, thread):
    while run.status=="queued" or run.status=="in_progress":
        run=client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [55]:
# 작업 완료시 Thread값 가져오는 함수
def get_response(thread_id):
    return client.beta.threads.messages.list(thread_id=thread_id, order="asc") # 최신질문, 답변이 맨 위로 올라오게 asc 설정

- JSON에서 필요한 내용만 추출하기
- 질문 + 답변
    - res.role.upper = User, Assistant 구분.
    - res.content[0].text.value = 질문, 답변.

In [56]:
def print_message(response):
    for res in response:
        print(f"[res.role.upper()]\n{res.content[0].text.value}\n")

In [64]:
# OpenAI의 JSON 값 전체 내용 확인할 때 사용
def show_json(obj):
    print(show_json(json.loads(obj.model_dump_json())))

### Main 함수

In [68]:
def main():
    #Threads id 생성
    THREAD_ID = create_new_thread().id
    
    while True:
        #assistant에게 전달할 질문내용
        question = input("Q: ")

        #종료 키워드
        if question.lower() in ["exit", "종료", "quit"]:
            print("종료합니다.")
            break

        #Thread에 메세지를 전송하고 Run실행
        run = summit_message(assistent_id, THREAD_ID, question)

        #Run상태 확인
        run1 = wait_on_run(run, THREAD_ID)

        #Run 확인 후 데이터 출력
        print_message(get_response(THREAD_ID).data[-2:])

    

In [66]:
main()

[res.role.upper()]
너에대해 소개해줘

[res.role.upper()]
저는 iOS 프로그래머로서 SwiftUI와 UIKit을 사용하여 앱 개발을 전문으로 하고 있습니다. 깔끔한 코드 작성에 중점을 두며, 효율적이고 유지보수가 용이한 코드를 작성하는 데 노력하고 있습니다. 여러 프로젝트를 통해 다양한 기술을 쌓아왔고, 최신 트렌드에 맞춘 개발 방식에 익숙합니다. 도움이 필요하시면 언제든지 질문해 주세요!

[res.role.upper()]
너에대해 소개해줘

[res.role.upper()]
저는 iOS 프로그래머로서 SwiftUI와 UIKit을 사용하여 앱 개발을 전문으로 하고 있습니다. 깔끔한 코드 작성에 중점을 두며, 효율적이고 유지보수가 용이한 코드를 작성하는 데 노력하고 있습니다. 여러 프로젝트를 통해 다양한 기술을 쌓아왔고, 최신 트렌드에 맞춘 개발 방식에 익숙합니다. 도움이 필요하시면 언제든지 질문해 주세요!

[res.role.upper()]
이전에 내가 질문했던 내역이있어?

[res.role.upper()]
이 대화의 이전 질문이나 내역은 저장되어 있지 않습니다. 현재 대화에서 주어진 정보만으로 도와드릴 수 있습니다. 궁금한 점이 있으면 언제든지 말씀해 주세요!

[res.role.upper()]
너에대해 소개해줘

[res.role.upper()]
저는 iOS 프로그래머로서 SwiftUI와 UIKit을 사용하여 앱 개발을 전문으로 하고 있습니다. 깔끔한 코드 작성에 중점을 두며, 효율적이고 유지보수가 용이한 코드를 작성하는 데 노력하고 있습니다. 여러 프로젝트를 통해 다양한 기술을 쌓아왔고, 최신 트렌드에 맞춘 개발 방식에 익숙합니다. 도움이 필요하시면 언제든지 질문해 주세요!

[res.role.upper()]
이전에 내가 질문했던 내역이있어?

[res.role.upper()]
이 대화의 이전 질문이나 내역은 저장되어 있지 않습니다. 현재 대화에서 주어진 정보만으로 도와드릴 수 있습니다. 궁금한 점이 있으면 언제든지 말씀

In [2]:
def chat():
    print("무엇을 도와드릴까요?")
    while True:
        
        question = input("Q: ")

        if question.lower() in ["exit", "quit", "종료"]:
            print("ChatGPT는 종료되었습니다.")
            break

        completion = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages=[{"role" : "developer",
                        "content" : """
                        You are a helpful assistant. 
                        You are a senior programmer. 
                        You are good at clean coding. 
                        You are Korean, Only Can anwser with Korean.
                        너는 한국인이야. 모든 답변은 반드시 한국어로만 대답해줘.
                        """}]
        )

        answer = completion.choices[0].message.content
        print(f"A: {answer}")

In [ ]:
chat()

무엇을 도와드릴까요?
A: 위 코드는 OpenAI의 API를 사용하여 사용자가 입력한 질문에 대해 답변하는 간단한 챗봇을 구현하려는 시도입니다. 하지만 몇 가지 문제가 있습니다. 이를 수정하고 설명을 추가하겠습니다.

1. **import 오류**: `import os`와 `import OpenAI`는 분리된 import 문으로 작성해야 합니다.
2. **API 키 로드**: 환경 변수를 사용하여 API 키를 로드하고 있어 잘 설정해주어야 합니다.
3. **모델 이름**: `gpt-4o-mini`라는 모델은 올바르지 않을 수 있습니다. 사용 가능한 모델 이름을 확인해 주세요.
4. **client 객체**: `OpenAI` 클라이언트 초기화가 코드에 없는 것 같습니다. `openai` 모듈을 사용하는 것이 일반적입니다.

아래는 수정된 버전입니다:

```python
import os
import openai
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# OpenAI API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

def chat():
    print("무엇을 도와드릴까요?")
    while True:
        question = input("Q: ")
        if question.lower() in ["exit", "quit"]:
            print("챗봇을 종료합니다.")
            break
        
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # 적절한 모델로 변경 (예: gpt-3.5-turbo, gpt-4 등)
            messages=[
                {"role": "user", "content": question}
 

In [ ]:


# load_dotenv()


# client = OpenAI()

# completion = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         #First "developer"
#         {
#             "role" : "developer", 
#             "content" : "You are helpful senior programmer assistant. you are Korean. only can speak Korean"
#         },
#         #Second "User"
#         {
#             "role" : "user",
#             "content" : "AWS 기업용 계정을 만드려면 어떻게 해야돼?"
#         }
#     ],
# )

In [12]:
# completion.choices[0].message

ChatCompletionMessage(content='AWS 기업용 계정을 만들기 위해서는 다음 단계를 따르세요:\n\n1. **AWS 홈페이지 방문**: [AWS 공식 웹사이트](https://aws.amazon.com/)에 접속합니다.\n\n2. **계정 생성 클릭**: 페이지 오른쪽 상단에서 "계정 만들기" 또는 "지금 시작하기" 버튼을 클릭합니다.\n\n3. **이메일 주소 입력**: 유효한 이메일 주소를 입력하고, 계정 이름과 비밀번호를 설정합니다. 이 때, 기업에 적합한 이메일 주소를 사용하는 것이 좋습니다.\n\n4. **계정 유형 선택**: 사용 목적에 따라 "개인" 또는 "기업" 계정을 선택합니다. 기업 계정을 선택하면 추가적인 정보 입력이 요구될 수 있습니다.\n\n5. **회사 정보 입력**: 회사의 이름, 주소, 전화번호 등 필요한 정보를 입력합니다.\n\n6. **결제 정보 입력**: 신용카드 정보를 입력하여 결제 수단을 설정합니다. AWS는 무료 사용량이 있지만, 유료 서비스 사용 시 결제가 필요하므로 신용카드 정보가 필요합니다.\n\n7. **정확성 확인 및 약관 동의**: 입력한 정보를 확인하고 AWS 서비스의 이용약관 및 개인정보 보호정책에 동의합니다.\n\n8. **인증 절차 완료**: AWS에서 이메일 인증 또는 전화 인증 절차가 있을 수 있습니다. 안내에 따라 인증을 완료합니다.\n\n9. **계정 대시보드 접근**: 인증이 완료되면 AWS 관리 콘솔에 로그인하여 서비스를 사용할 수 있습니다.\n\n이런 과정을 통해 AWS 기업용 계정을 생성할 수 있습니다. 추가적으로, 기업의 요구에 맞게 IAM(Identity and Access Management)를 통해 사용자 및 권한 관리를 하는 것이 중요합니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)